In [ ]:
%pip install tableone

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tableone
from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='datathon')

In [ ]:
%%bigquery df_o2 --project datathon-455912

SELECT *
FROM `physionet-data.eicu_crd_derived.pivoted_o2`
WHERE
  (
    LOWER(o2_device) IN (
      'ventilator', 'nasal cannula', 'ra', 'bipap/cpap', 'trach collar', 'room air',
      'venturi mask', 'nc', 'non-rebreather', 'cool aerosol mask', 'vent', 'other',
      'bipap', 'hfnc', 'oxymizer', 'cpap'
    )
    OR o2_device IS NULL
  )
ORDER BY patientunitstayid, chartoffset

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_o2

,patientunitstayid,chartoffset,entryoffset,o2_flow,o2_device,etco2
0,141168,506,513,2.0,nasal cannula,NaN
1,142723,1137,1124,60.0,ventilator,NaN
2,143325,605,632,30.0,ventilator,NaN
3,143325,1685,1713,60.0,ventilator,NaN
4,144297,10421,10426,60.0,ventilator,NaN
...,...,...,...,...,...,...
3019424,3353263,530,530,2.0,None,NaN
3019425,3353263,590,590,2.0,None,NaN
3019426,3353263,650,650,2.0,None,NaN
3019427,3353263,710,710,2.0,None,NaN


In [ ]:
# Group by device group and show min, median, max of o2_flow
flow_summary = (
    df_o2
    .groupby('o2_device_group')['o2_flow']
    .agg(['count', 'min', 'median', 'max', 'mean', 'std'])
    .sort_values(by='count', ascending=False)
)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)

print("✅ Oxygen Flow Summary per Device Group:")
print(flow_summary)


✅ Oxygen Flow Summary per Device Group:
                  count  min  median    max       mean        std
o2_device_group                                                  
ventilator       499660  0.0    50.0  100.0  49.573570  17.783838
nasal cannula    455929  0.0    30.0  100.0  23.301739  18.536303
room air         160801  0.0    21.0  100.0  20.354620  15.567352
mask oxygen       77399  0.0    50.0  100.0  47.428185  19.388978
bipap/cpap        59397  0.0    40.0  100.0  41.445260  20.884072
other              6810  0.0    30.0  100.0  31.142070  20.561744
hfnc               5444  0.5    50.0  100.0  42.060470  31.466427


In [ ]:
# Check missing o2_device_group
print(f"✅ Number of missing o2_device_group: {df_o2['o2_device_group'].isnull().sum()}")

# Check missing o2_flow and o2_device_group together
missing_both = df_o2[df_o2['o2_flow'].isnull() & df_o2['o2_device_group'].isnull()]
print(f"✅ Number of rows missing BOTH o2_flow and o2_device_group: {len(missing_both)}")

# Look at them
print(missing_both[['patientunitstayid', 'chartoffset', 'o2_device', 'o2_flow']].head(20))


✅ Number of missing o2_device_group: 1753989
✅ Number of rows missing BOTH o2_flow and o2_device_group: 130864
      patientunitstayid  chartoffset o2_device  o2_flow
9                146069          155      None      NaN
53               179813           62      None      NaN
1759             244396           11      None      NaN
2298             244975           86      None      NaN
2437             245347         1793      None      NaN
2461             245542          101      None      NaN
2462             245542          341      None      NaN
3728             246953          340      None      NaN
3729             246953          580      None      NaN
3730             246953          820      None      NaN
3731             246953          880      None      NaN
3732             246953         1060      None      NaN
3733             246953         1300      None      NaN
3734             246953         1360      None      NaN
3735             246953         1540      None   

In [ ]:
# Step 1: Identify rows where both o2_device_group and o2_flow are missing
both_missing_mask = df_o2['o2_device_group'].isnull() & df_o2['o2_flow'].isnull()

print(f"✅ Number of rows to remove (both missing): {both_missing_mask.sum()}")

# Step 2: Drop those rows
df_o2 = df_o2[~both_missing_mask].reset_index(drop=True)

print(f"✅ After dropping, df_o2 shape: {df_o2.shape}")
print(f"✅ Remaining missing o2_flow: {df_o2['o2_flow'].isnull().sum()}")

✅ Number of rows to remove (both missing): 0
✅ After dropping, df_o2 shape: (2888565, 8)
✅ Remaining missing o2_flow: 0


In [ ]:
## Remove the unrealistic values

device_flow_range = {
    'room air': (0, 30),                # Room air should be close to 0–30 (21% FiO₂)
    'nasal cannula': (0, 50),            # low flow
    'mask oxygen': (0, 70),              # trach collar, venturi, etc.
    'non-rebreather': (40, 100),         # high flow masks
    'bipap/cpap': (30, 100),             # noninvasive ventilation
    'hfnc': (30, 100),                   # high flow nasal cannula
    'ventilator': (30, 100),             # mechanical ventilation
    'other': (0, 100),                   # allow full range
    np.nan: (0, 100)                     # <<< add missing device group explicitly
}

def is_flow_realistic(row):
    device = row['o2_device_group']
    flow = row['o2_flow']

    if pd.isna(flow):
        return False  # Missing flow cannot be realistic

    flow_range = device_flow_range.get(device, (0, 100))  # fallback
    return flow_range[0] <= flow <= flow_range[1]

# Apply the function to each row
df_o2['realistic_flow'] = df_o2.apply(is_flow_realistic, axis=1)

# Check results
print(df_o2['realistic_flow'].value_counts())


realistic_flow
True     2834369
False      54196
Name: count, dtype: int64


In [ ]:
# Filter to keep only realistic rows
df_o2_cleaned = df_o2[df_o2['realistic_flow']].copy()

# Drop the helper column
df_o2_cleaned = df_o2_cleaned.drop(columns=['realistic_flow'])

# Check final shape
print(f"✅ Final cleaned df_o2 shape: {df_o2_cleaned.shape}")

✅ Final cleaned df_o2 shape: (2834369, 7)


In [ ]:
# Step 1: If device is missing, assume nasal cannula
df_o2['o2_device_group'] = df_o2['o2_device_group'].fillna('nasal cannula')

# Step 2: Remove nasal cannula rows with flow between 6–20
mask_suspect_flow = (
    (df_o2['o2_device_group'] == 'nasal cannula') &
    (df_o2['o2_flow'] > 6) &
    (df_o2['o2_flow'] <= 20)
)

print(f"✅ Number of nasal cannula rows with flow 6–20 to drop: {mask_suspect_flow.sum()}")

# Drop them
df_o2 = df_o2[~mask_suspect_flow].reset_index(drop=True)

# Step 3: Define FiO₂ estimation function
def estimate_fio2(row):
    device = row['o2_device_group']
    flow = row['o2_flow']

    if pd.isna(flow):
        return np.nan  # can't estimate without flow

    if device == 'nasal cannula':
        if flow <= 1:
            return 24
        elif flow <= 2:
            return 28
        elif flow <= 3:
            return 32
        elif flow <= 4:
            return 36
        elif flow <= 5:
            return 40
        elif flow <= 6:
            return 44
        else:
            return 50  # this case now should be rare after drop
    else:
        return np.nan  # Other devices not estimated here

# Step 4: Apply to get estimated FiO2
df_o2['estimated_fio2'] = df_o2.apply(estimate_fio2, axis=1)

# Step 5: Look at results
print("✅ Estimated FiO₂ for nasal cannula calculated:")
print(df_o2[['patientunitstayid', 'chartoffset', 'o2_device', 'o2_flow', 'estimated_fio2']].head(30))


✅ Number of nasal cannula rows with flow 6–20 to drop: 119112
✅ Estimated FiO₂ for nasal cannula calculated:
    patientunitstayid  chartoffset      o2_device  o2_flow  estimated_fio2
0              141168          506  nasal cannula      2.0            28.0
1              142723         1137     ventilator     60.0             NaN
2              143325          605     ventilator     30.0             NaN
3              143325         1685     ventilator     60.0             NaN
4              144297        10421     ventilator     60.0             NaN
5              144297        17681     ventilator     60.0             NaN
6              144297        59381     ventilator     40.0             NaN
7              144774          348           None     80.0            50.0
8              145715         2277     ventilator     70.0             NaN
9              146945            1           None      2.0            28.0
10             153972            6           None     21.0        

In [ ]:
# Keep only essential columns
df_o2_final = df_o2[['patientunitstayid', 'chartoffset', 'entryoffset', 'o2_device_group', 'final_fio2']].copy()

# Check
print(f"✅ Final cleaned df shape: {df_o2_final.shape}")
print(df_o2_final.head(10))

✅ Final cleaned df shape: (2769453, 5)
   patientunitstayid  chartoffset  entryoffset o2_device_group  final_fio2
0             141168          506          513   nasal cannula        28.0
1             142723         1137         1124      ventilator        60.0
2             143325          605          632      ventilator        30.0
3             143325         1685         1713      ventilator        60.0
4             144297        10421        10426      ventilator        60.0
5             144297        17681        17658      ventilator        60.0
6             144297        59381        59401      ventilator        40.0
7             144774          348          348   nasal cannula        50.0
8             145715         2277         2253      ventilator        70.0
9             146945            1           13   nasal cannula        28.0
